In [1]:
import pyspark
import dxpy
import hail as hl
import gzip

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)
hl.init(sc=sc, default_reference='GRCh38')

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-63-196.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/notebooks/hail-20211110-1439-0.2.61-3c86d3ba497a.log


In [3]:
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover("file:" + "/opt/notebooks/gogoGPCR/tmp/hg19ToHg38.over.filtered.chain.gz", rg38)

In [4]:
mt = hl.import_vcf("file:" + "/mnt/project/Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants\,\ pVCF\ format/ukb23156_c13_b7_v1.vcf.gz",
                   force_bgz = True,
                   array_elements_required = False,
                   drop_samples = True)

In [6]:
mt.checkpoint("/tmp", overwrite = True)

2021-11-10 14:44:32 Hail: INFO: Coerced sorted dataset
2021-11-10 14:46:00 Hail: INFO: wrote matrix table with 13687 rows and 0 columns in 46 partitions to /tmp
    Total size: 327.41 KiB
    * Rows/entries: 327.39 KiB
    * Columns: 10.00 B
    * Globals: 11.00 B
    * Smallest partition: 177 rows (4.05 KiB)
    * Largest partition:  428 rows (10.05 KiB)


In [7]:
mt.show()

2021-11-10 14:49:10 Hail: INFO: Coerced sorted dataset


,
locus,alleles
locus<GRCh38>,array<str>
chr10:46385740,"[""C"",""G""]"
chr10:46385767,"[""T"",""C""]"
chr10:46385824,"[""G"",""A""]"
chr10:46385827,"[""C"",""T""]"
chr10:46385828,"[""G"",""A""]"
chr10:46385852,"[""G"",""A""]"
chr10:46385855,"[""G"",""A""]"
chr10:46385857,"[""G"",""A""]"


In [11]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)
        

writing = True

with gzip.open("/opt/notebooks/gogoGPCR/tmp/hg19ToHg38.over.filtered.chain.gz", "wt") as new:
    for f in gzip.open("/opt/notebooks/gogoGPCR/tmp/hg19ToHg38.over.chain.gz", "rt"):
        if f.startswith("chain") and (("_" in f) or ("M" in f)):
            writing = False
        if f.startswith("chain") and (("_" not in f) and ("M" not in f)):
            writing = True
            f = f.replace("chr", "", 1)
            
        if writing:
            new.write(f)
        
    
    

In [6]:
str.replace?

Docstring:
S.replace(old, new[, count]) -> str

Return a copy of S with all occurrences of substring
old replaced by new.  If the optional argument count is
given, only the first count occurrences are replaced.
Type:      method_descriptor


In [ ]:
mt.count()

2021-11-10 14:53:39 Hail: INFO: Coerced sorted dataset
